# Example: Adversarial Figure 8 with 2 Lanes of Traffic

This example walks you through implementing modifications in the figure 8 template. In particular, we show how to modify the number of lanes, change the car behaviour accordingly and add an adversarial agent that learns how to slow down the flow of traffic using RL. We explain the settings used to modify each parameter so you can configure them based on your own needs.


<img src="img/fig8_2lane_crowding.png" width="450" height="450">

<center>**Figure 1.** Traffic on a figure 8 environment with two lanes and an adversarial agent</center>

The remainder of this tutorial is organized as follows:
 
* We start with an overview of the Figure 8 example from the core files 
* Section 1 gives an overview of the scripts involved in the simulation
* Section 2 explains the changes on the environment file
* Section 3 explains the changes on the network file
* Section 4 shows how to run the training script


## The Figure 8 example
* The goal is to maximize the system-wide velocity for fourteen vehicles, which necessitates spacing the vehicles so that they don’t run into conflicts at the merging points. 

* The network is fully observed: all vehicles speeds and positions are visible to the controller. 

* This is an example of a multi-agent adversarial environment. 
* The example consists of one autonomous vehicle controlled by 2 agents:

1. an RL-trained adversary agent whose objective is to disrupt the flow of traffic. 
2. an optimizing RL agent ('AV vehicle') whose objective is to optimize the flow of traffic (i.e. maximizing the speed of traffic)

* Note that the expected total reward is zero as the adversary's reward is the negative of the AV reward

* The example on this tutorial extends the simple figure8 example described here:

    https://flow.readthedocs.io/en/latest/examples.html#figure-eight
    
    https://flow.readthedocs.io/en/latest/examples.html

    https://flow-project.github.io/papers/vinitsky18a.pdf






## 1. Overview of the Steps Required

The idea is to reuse the existing code and modify it according to our needs. 
Below are the steps required and the files we need to modify:

**1. Change the configuration file to avoid script stopping at errors.**

    ~flow/examples/exp_configs/rl/multiagent/multiagent_figure_eight.py

**2. Change the configuration file to increase the the weight of the adversarial agent in the AV car**

    ~flow/examples/exp_configs/rl/multiagent/multiagent_figure_eight.py
    
**3. Change the controllers dynamics to allow human and AV cars to pass each other.**

    ~flow/examples/exp_configs/rl/multiagent/multiagent_figure_eight.py
    

**4. Change the network file to allow 2 lanes.**

    ~flow/flow/networks/figure_eight.py


**5. Run the usual runner used to train.**

    ~flow/examples/train.py

## 2. Changes in the Configuration File

File path:
**~flow/examples/exp_configs/rl/multiagent/multiagent_figure_eight.py**

### 2.1 Add More Human Vehicles:
We will change the number of human vehicles a piaccere, this is to show clearly the effects of a lane change. As an example, change the number of Human cars as follows:

**N_HUMANS   = 25**

### 2.2 Changes in SUMO-Related Parameters:
We will add this line, to avoid the script stopping at errors:

**restart_instance = True**

### 2.3 Add Lane Change Controllers to Human and AV Vehicles
When we add a second line to our figure eight, we would like our cars to benefit from the additional line to pass each other to optimize the flow of traffic. We will import the controller and then add the controlling parameters to both the human and autonomous vehicles

* Add the following import:

    **from flow.controllers import SimLaneChangeController**

* Add the following parameters to the humand and the av agents

    **lane_change_controller=(SimLaneChangeController, {}),**
    **lane_change_params=SumoLaneChangeParams(lane_change_mode="strategic",),**


#### **Note: Additional Information on Line Changing Modes**
The line changing parameters are controlled by SUMO. 
In the line above we have used the 'lane_change_mode="strategic"'

You can read more about the available options from SUMO documentation:

The laneChangeModel discriminates four reasons to change lanes:

    * strategic (change lanes to continue the route)
    * cooperative (change in order to allow others to change)
    * speed gain (the other lane allows for faster driving)
    * obligation to drive on the right


https://sumo.dlr.de/docs/TraCI/Change_Vehicle_State.html#lane_change_mode_0xb6

https://sumo.dlr.de/docs/Definition_of_Vehicles,_Vehicle_Types,_and_Routes.html#lane-changing_models

https://sumo.dlr.de/docs/Simulation/SublaneModel.html#lane-changing

### 2.4 Increasing the weight of the adversarial agent.
Change the perturbation weight to see the adversarial agent in full control

**'perturb_weight': 0.9**
    

### 2.5 Changes in the Configuration File: Putting It All Together...
Your environment file should look like this:

In [ ]:
#~flow/examples/exp_configs/rl/multiagent/multiagent_figure_eight.py

    from copy import deepcopy
    from ray.rllib.agents.ppo.ppo_policy import PPOTFPolicy
    from flow.controllers import ContinuousRouter
    from flow.controllers import IDMController #the human vehicles follows a physics algo
    from flow.controllers import RLController  #the av drives according to RL algo
    from flow.controllers import SimLaneChangeController #Controller used to enforce sumo lane-change dynamics on a vehicle.
    from flow.core.params import EnvParams
    from flow.core.params import InitialConfig
    from flow.core.params import NetParams
    from flow.core.params import SumoParams
    from flow.core.params import SumoCarFollowingParams,SumoLaneChangeParams #lane change params
    from flow.core.params import VehicleParams
    from flow.networks.figure_eight import ADDITIONAL_NET_PARAMS #specifies the env structure (for example, the number of lanes)
    from flow.envs.multiagent import MultiAgentAccelEnv
    from flow.networks import FigureEightNetwork
    from flow.utils.registry import make_create_env
    from ray.tune.registry import register_env

    # time horizon of a single rollout
    HORIZON    = 1500
    # number of rollouts per training iteration
    N_ROLLOUTS = 4
    # number of parallel workers
    N_CPUS     = 44
    # number of human-driven vehicles
    N_HUMANS   = 25 #increased number of human cars to make it more dramatic
    # number of automated vehicles
    N_AVS      = 1

     # ContinuousRouter controller -> to perpetually maintain the vehicle within the network.
    # lane_change_controller=(SimLaneChangeController, {}) -> used to enforce sumo lane-change dynamics on a vehicle.
    # lane_change_params=SumoLaneChangeParams(lane_change_mode="strategic",) - > cars can change lane

    vehicles = VehicleParams()
    vehicles.add(
        veh_id='human',
        lane_change_controller=(SimLaneChangeController, {}),
        lane_change_params=SumoLaneChangeParams(lane_change_mode="strategic",),
        acceleration_controller=(IDMController, {
            'noise': 0.2
        }),
        routing_controller=(ContinuousRouter, {}),
        car_following_params=SumoCarFollowingParams(
            speed_mode='obey_safe_speed',
        ),
        num_vehicles=N_HUMANS)

    #RL agent
    vehicles.add(
        veh_id='rl',
        lane_change_controller=(SimLaneChangeController, {}),
        lane_change_params=SumoLaneChangeParams(lane_change_mode="strategic",),
        acceleration_controller=(RLController, {}),
        routing_controller=(ContinuousRouter, {}),
        car_following_params=SumoCarFollowingParams(
            speed_mode='obey_safe_speed',
        ),
        num_vehicles=N_AVS)

    flow_params = dict(
        # name of the experiment
        exp_tag='multiagent_figure_eight',

        # name of the flow environment the experiment is running on
        env_name=MultiAgentAccelEnv,

        # name of the network class the experiment is running on
        network=FigureEightNetwork,

        # simulator that is used by the experiment
        simulator='traci',

        # sumo-related parameters (see flow.core.params.SumoParams)
        sim=SumoParams(
            sim_step =0.1,
            render   =False,
            restart_instance = True,
         ),

        # environment related parameters (see flow.core.params.EnvParams)
        env=EnvParams(
            horizon=HORIZON,
            additional_params={
                'target_velocity': 20,
                'max_accel': 3,
                'max_decel': 3,
                'perturb_weight': 0.9, #0.03, #weight of the adversarial agent
                'sort_vehicles': False
            },
        ),

        # network-related parameters (see flow.core.params.NetParams and the
        # network's documentation or ADDITIONAL_NET_PARAMS component)
        net=NetParams(
            additional_params=deepcopy(ADDITIONAL_NET_PARAMS),
        ),

        # vehicles to be placed in the network at the start of a rollout (see
        # flow.core.params.VehicleParams)
        veh=vehicles,

        # parameters specifying the positioning of vehicles upon initialization/
        # reset (see flow.core.params.InitialConfig)
        initial=InitialConfig(),
    )


    create_env, env_name = make_create_env(params=flow_params, version=0)

    # Register as rllib env
    register_env(env_name, create_env)

    test_env = create_env()
    obs_space = test_env.observation_space
    act_space = test_env.action_space


    def gen_policy():
        """Generate a policy in RLlib."""
        return PPOTFPolicy, obs_space, act_space, {}


    # Setup PG with an ensemble of `num_policies` different policy graphs
    POLICY_GRAPHS = {'av': gen_policy(), 'adversary': gen_policy()}


    def policy_mapping_fn(agent_id):
        """Map a policy in RLlib."""
        return agent_id

## 3. Changes in the Network File
We will change the network file to allow 2 lanes.

File path: **~flow/flow/networks/figure_eight.py**

Change the "lanes" entry in the ADDITIONAL_NET_PARAMS dictionary as shown below


In [4]:
ADDITIONAL_NET_PARAMS = {
    # radius of the circular components
    "radius_ring": 30,
    # number of lanes
    "lanes": 2,  #CHANGE HERE to add more lanes to the environment
    # speed limit for all edges
    "speed_limit": 30,
    # resolution of the curved portions
    "resolution": 40
}


## 4. Runner

After implementing the above changes, you can run the trainimng script as before

In [ ]:
%run ~flow/examples/train multiagent_figure_eight

In [3]:
#“Life is not easy for any of us. But what of that? We must have perseverance and above all have confidence in ourselves. We must believe that we are gifted for something and that this thing must be attained.” Madame Curie